<a href="https://colab.research.google.com/github/soyyoon/Sign2Talk/blob/main/DATA_PREPROCESSING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 123.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 12.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled 

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os
import glob
from tqdm import tqdm
from google.colab import drive
drive.mount('/content/drive')

mp_holistic = mp.solutions.holistic

/usr/local/lib/python3.12/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.7.2 is installed, but it is not compatible with the installed jaxlib version 0.7.1, so it will not be used.
  warnings.warn(


Mounted at /content/drive


In [ ]:
def calculate_angle(v1, v2):
    """두 벡터 사이의 각도 계산"""
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)

    if norm_v1 == 0 or norm_v2 == 0:
        return 0

    cos_angle = dot_product / (norm_v1 * norm_v2)
    cos_angle = np.clip(cos_angle, -1.0, 1.0)
    return np.arccos(cos_angle)


def calculate_distance(p1, p2):
    """유클리드 거리"""
    return np.sqrt(np.sum((np.array(p1) - np.array(p2)) ** 2))


def extract_basic_landmarks(video_path, target_frames=30):
    """기본 47차원 특징만 추출"""
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        return None

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames > target_frames:
        frame_indices = np.linspace(0, total_frames - 1, target_frames, dtype=int)
    else:
        frame_indices = list(range(total_frames))

    features_sequence = []

    with mp_holistic.Holistic(
        static_image_mode=False,
        model_complexity=1,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    ) as holistic:

        frame_idx = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            if frame_idx not in frame_indices:
                frame_idx += 1
                continue

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = holistic.process(frame_rgb)

            frame_features = extract_frame_features(results)

            # ⭐ 수정: 정확히 47차원인지 확인
            if len(frame_features) == 47:
                features_sequence.append(frame_features)
            else:
                # 길이가 다르면 0으로 채움
                features_sequence.append(np.zeros(47))

            frame_idx += 1

            if len(features_sequence) >= target_frames:
                break

    cap.release()

    # Padding
    while len(features_sequence) < target_frames:
        features_sequence.append(np.zeros(47))

    # ⭐ 수정: numpy array로 변환 전에 모든 요소가 같은 shape인지 확인
    features_array = np.array(features_sequence[:target_frames], dtype=np.float32)

    return features_array


def extract_frame_features(results):
    """47차원 특징 추출 (정확히 47개 보장)"""
    features = []

    # 왼손 (10차원)
    if results.left_hand_landmarks:
        left_features = extract_hand_features(results.left_hand_landmarks.landmark)
        features.extend(left_features[:10])
    else:
        features.extend([0] * 10)

    # 오른손 (10차원)
    if results.right_hand_landmarks:
        right_features = extract_hand_features(results.right_hand_landmarks.landmark)
        features.extend(right_features[:10])
    else:
        features.extend([0] * 10)

    # 양손 상호작용 (15차원)
    if results.left_hand_landmarks and results.right_hand_landmarks:
        interaction_features = extract_hand_interaction(
            results.left_hand_landmarks.landmark,
            results.right_hand_landmarks.landmark
        )
        features.extend(interaction_features[:15])
    else:
        features.extend([0] * 15)

    # 포즈 (12차원)
    if results.pose_landmarks:
        pose_features = extract_pose_features(results.pose_landmarks.landmark)
        features.extend(pose_features[:12])
    else:
        features.extend([0] * 12)

    # ⭐ 수정: 정확히 47개 보장
    if len(features) < 47:
        features.extend([0] * (47 - len(features)))
    elif len(features) > 47:
        features = features[:47]

    return np.array(features, dtype=np.float32)


def extract_hand_features(landmarks):
    """손 특징 (정확히 10차원)"""
    features = []

    wrist = 0
    thumb_tip = 4
    index_tip = 8
    middle_tip = 12
    ring_tip = 16
    pinky_tip = 20

    # 손가락 각도 (5차원)
    for tip in [thumb_tip, index_tip, middle_tip, ring_tip, pinky_tip]:
        v1 = np.array([landmarks[wrist].x - landmarks[tip].x,
                      landmarks[wrist].y - landmarks[tip].y])
        v2 = np.array([0, -1])
        angle = calculate_angle(v1, v2)
        features.append(float(angle))

    # 손가락 간 거리 (5차원)
    tips = [thumb_tip, index_tip, middle_tip, ring_tip, pinky_tip]
    for i in range(len(tips) - 1):
        p1 = [landmarks[tips[i]].x, landmarks[tips[i]].y]
        p2 = [landmarks[tips[i + 1]].x, landmarks[tips[i + 1]].y]
        dist = calculate_distance(p1, p2)
        features.append(float(dist))

    return features[:10]  # ⭐ 정확히 10개만 반환


def extract_hand_interaction(left_landmarks, right_landmarks):
    """양손 상호작용 (정확히 15차원)"""
    features = []

    key_points = [0, 4, 8, 12, 16, 20]

    # 양손 간 거리 (6차원)
    for idx in key_points:
        p1 = [left_landmarks[idx].x, left_landmarks[idx].y]
        p2 = [right_landmarks[idx].x, right_landmarks[idx].y]
        dist = calculate_distance(p1, p2)
        features.append(float(dist))

    # 손목 거리 (1차원)
    left_wrist = [left_landmarks[0].x, left_landmarks[0].y]
    right_wrist = [right_landmarks[0].x, right_landmarks[0].y]
    wrist_dist = calculate_distance(left_wrist, right_wrist)
    features.append(float(wrist_dist))

    # 손 방향 (8차원)
    for hand_lm in [left_landmarks, right_landmarks]:
        wrist = [hand_lm[0].x, hand_lm[0].y]
        middle = [hand_lm[9].x, hand_lm[9].y]

        direction = np.array([middle[0] - wrist[0], middle[1] - wrist[1]])

        for ref_dir in [[0, -1], [1, 0], [0, 1], [-1, 0]]:
            angle = calculate_angle(direction, ref_dir)
            features.append(float(angle))

    return features[:15]


def extract_pose_features(landmarks):
    """포즈 특징 (정확히 12차원)"""
    features = []

    # 팔 각도 (2차원)
    for shoulder, elbow, wrist in [(11, 13, 15), (12, 14, 16)]:
        v1 = np.array([landmarks[elbow].x - landmarks[shoulder].x,
                      landmarks[elbow].y - landmarks[shoulder].y])
        v2 = np.array([landmarks[wrist].x - landmarks[elbow].x,
                      landmarks[wrist].y - landmarks[elbow].y])
        angle = calculate_angle(v1, v2)
        features.append(float(angle))

    # 몸통 대비 팔 위치 (8차원)
    shoulder_center_x = (landmarks[11].x + landmarks[12].x) / 2
    shoulder_center_y = (landmarks[11].y + landmarks[12].y) / 2
    hip_center_x = (landmarks[23].x + landmarks[24].x) / 2
    hip_center_y = (landmarks[23].y + landmarks[24].y) / 2

    for wrist_idx in [15, 16]:
        features.append(float(landmarks[wrist_idx].x - shoulder_center_x))
        features.append(float(landmarks[wrist_idx].y - shoulder_center_y))
        features.append(float(landmarks[wrist_idx].x - hip_center_x))
        features.append(float(landmarks[wrist_idx].y - hip_center_y))

    # 어깨 너비 (1차원)
    shoulder_width = calculate_distance(
        [landmarks[11].x, landmarks[11].y],
        [landmarks[12].x, landmarks[12].y]
    )
    features.append(float(shoulder_width))

    # 몸통 높이 (1차원)
    torso_height = calculate_distance(
        [shoulder_center_x, shoulder_center_y],
        [hip_center_x, hip_center_y]
    )
    features.append(float(torso_height))

    return features[:12]

In [ ]:
def process_videos_simple(video_dir, output_dir, target_frames=30, overwrite=False):
    """비디오 처리 메인 함수"""
    os.makedirs(output_dir, exist_ok=True)

    class_folders = sorted([d for d in os.listdir(video_dir)
                           if os.path.isdir(os.path.join(video_dir, d))
                           and not d.startswith('.')])

    print(f"📂 Found {len(class_folders)} classes")
    print(f"🎯 Target frames: {target_frames}")
    print(f"💾 Output directory: {output_dir}")

    total_processed = 0
    total_failed = 0

    for class_name in tqdm(class_folders, desc="Processing classes"):
        class_input_dir = os.path.join(video_dir, class_name)
        class_output_dir = os.path.join(output_dir, class_name)
        os.makedirs(class_output_dir, exist_ok=True)

        video_files = []
        video_files += glob.glob(os.path.join(class_input_dir, "*.mp4"))
        video_files += glob.glob(os.path.join(class_input_dir, "*.MP4"))  # 대문자 추가
        video_files += glob.glob(os.path.join(class_input_dir, "*.avi"))
        video_files += glob.glob(os.path.join(class_input_dir, "*.AVI"))  # 대문자 추가

        for video_path in video_files:
            video_name = os.path.basename(video_path)
            output_name = os.path.splitext(video_name)[0] + ".npy"
            output_path = os.path.join(class_output_dir, output_name)

            if os.path.exists(output_path) and not overwrite:
                total_processed += 1  # 이미 처리됨으로 카운트
                continue

            features = extract_basic_landmarks(video_path, target_frames)

            if features is not None:
                np.save(output_path, features)
                total_processed += 1
            else:
                total_failed += 1

    print(f"\n✅ Processing complete!")
    print(f"   Processed: {total_processed}")
    print(f"   Failed: {total_failed}")

    return {'total_processed': total_processed, 'total_failed': total_failed}

In [ ]:
VIDEO_DIR = '/content/drive/MyDrive/S2T/KSL_ACTION_VIDEO'
OUTPUT_DIR = '/content/drive/MyDrive/S2T/features_basic_47dim'

print("="*60)
print("🎬 비디오 전처리 시작")
print("="*60)

results = process_videos_simple(
    video_dir=VIDEO_DIR,
    output_dir=OUTPUT_DIR,
    target_frames=30,
    overwrite=True  # 기존 빈 폴더 다시 채우기
)

print(f"\n🎉 Done! Features saved to: {OUTPUT_DIR}")

# 결과 확인
print("\n" + "="*60)
print("📊 결과 확인")
print("="*60)

class_folders = [d for d in os.listdir(OUTPUT_DIR)
                 if os.path.isdir(os.path.join(OUTPUT_DIR, d))]

if class_folders:
    sample_class = os.path.join(OUTPUT_DIR, class_folders[0])
    npy_files = glob.glob(os.path.join(sample_class, "*.npy"))

    print(f"\n클래스 '{class_folders[0]}': {len(npy_files)}개 파일")

    if npy_files:
        sample = np.load(npy_files[0])
        print(f"샘플 shape: {sample.shape}")

        if sample.shape == (30, 47):
            print("✅ 완벽합니다! 학습 시작 가능!")
        else:
            print(f"⚠️ Shape이 예상과 다릅니다")

🎬 비디오 전처리 시작
📂 Found 67 classes
🎯 Target frames: 30
💾 Output directory: /content/drive/MyDrive/S2T/features_basic_47dim


Processing classes: 100%|██████████| 67/67 [1:25:19<00:00, 76.41s/it]


✅ Processing complete!
   Processed: 1229
   Failed: 0

🎉 Done! Features saved to: /content/drive/MyDrive/S2T/features_basic_47dim

📊 결과 확인

클래스 '01': 20개 파일
샘플 shape: (30, 47)
✅ 완벽합니다! 학습 시작 가능!
